In [ ]:
import sys
sys.path.insert(0, '/projekti/mondoAPI')
sys.path.insert(0, '/projekti/wikidata')

import json
from collections import Counter, defaultdict
from wiki_labels import qid_lab_get
from wikilang2iso import get_wiki_cc
from text_utils import cl
from pnu.parse import parse
from api.db import db
from tqdm.notebook import tqdm

qid_lab_get(42, 'en').keys(), get_wiki_cc('country': ['Q161885']), parse('davor lauc')['tags'][0]

In [ ]:
fo = open('/backup/wikidata/wiki_person.tsv', 'w')
fo.write('qid\tname\tfn\tln\tdesc\tplace\tdob\timage\tsort\n')
for i, l in tqdm(enumerate(open('/backup/wikidata/wiki_person.jsonl')), total=100_000_000):
    j = json.loads(l)
    qid = int(j['id'][1:])
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    name = None
    if j['name_native']:
        name = j['name_native'][0]
    else:
        if j['language']:
            langs = Counter([iso2w[q[5:]] for q in j['language'] if q[5:] in iso2w])
        else:
            langs = Counter()
        langs.update(cc2lang[cc])
        for l, f in langs.most_common():
            if l in j['l']:
                name = j['l'][l][0]
                break
    if not name:
        try:
            name = list(qid_lab_get(qid, 'en', include_alt=False).keys())[0]
        except:
            try:
                name = list(j['l'].values())[0][0]
            except:
                name = ''
    if name:
        namel = name.lower().strip()
        try:
            desc = j['desc']['en'][0]
        except:
            desc = ''
        dob = j['dob'][0][:4] if j['dob'] else ''
        image = j['picture'][0] if j['picture'] else ''
        sort = (int(dob) if dob else 1980)-(50 if image else 0)-(10 if desc else 0)
        
        fn = ''
        if j['fn']:
            fns = qid_lab_get(int(j['fn'][0][6:]), include_alt=True).keys()
            fns = Counter({a:len(a) for a in fns if a in namel})
            if len(fns)>0:
                fn = fns.most_common()[0][0]

        ln = ''
        if j['ln']:
            fns = qid_lab_get(int(j['ln'][0][6:]), include_alt=True).keys()
            fns = Counter({a:len(a) for a in fns if a in namel})
            if len(fns)>0:
                ln = fns.most_common()[0][0]
        if not fn or not ln:
            if ',' in namel:
                if ' ' in namel.split(',')[0].strip():
                    namel = namel.split(',')[0].strip()
                else:
                    namel = namel.split(',')[1].strip()+' '+namel.split(',')[0].strip()
                #print(name, namel)
            parts = namel.split(' ')
            if len(parts) == 1 and fn:
                if namel.startswith(fn):
                    parts = [fn, namel[len(fn):]]
                elif namel.endswith(fn):
                    parts = [fn, namel[:len(namel)-len(fn)]]
            if len(parts) == 1 and ln:
                if namel.startswith(ln):
                    parts = [namel[len(ln):], ln]
                elif namel.endswith(ln):
                    parts = [namel[:len(namel)-len(ln)], ln]
                
            if len(parts) == 2:
                if fn and parts[0]==fn:
                    ln = parts[1]
                elif fn and parts[1]==fn:
                    ln = parts[0]
                elif ln and parts[1]==ln:
                    fn = parts[0]
                elif ln and parts[0]==ln:
                    fn = parts[1]
                else:
                    fn, ln = parts
            elif len(parts) > 2:
                fn, ln = parts[0], parts[-1]
                    
        fo.write(f'{qid}\t{name}\t{fn}\t{ln}\t{desc}\t{cc}\t{dob}\t{image}\t{sort}\n')
fo.close()
    

  0%|          | 0/100000000 [00:00<?, ?it/s]

In [99]:
ln

'coghetto'

In [57]:
fo = open('/backup/wikidata/wiki_parsed.tsv', 'w')
for i, l in tqdm(enumerate(open('/backup/wikidata/wiki_person.jsonl')), total=100_000_000):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if j['language']:
        langs = [iso2w[q[5:]] for q in j['language'] if q[5:] in iso2w]
    elif cc:
        langs = cc2lang[cc]
    langs = list(set(langs))
    names = []
    for l in langs:
        if l in j['l']:
            for name in j['l'][l]:
                names.append(name)
    names = set(list(names))

    if not cc:
        cc = ''
    for name in names:
        rec = parse(name)
        titles, fn, ln = [], [], []
        prob = -1 
        guessCC = ''
        if rec['tags'] and rec['tags'][0]['prob']>0.2:
            prob = rec['tags'][0]['prob']
            for tok, typ in zip(rec['tags'][0]['tokens'], rec['tags'][0]['labels']):
                if typ[:5] == 'title':
                    titles.append(tok)
                elif typ[:2] == 'fn':
                    fn.append(tok)
                elif typ[:2] == 'ln':
                    ln.append(tok)
                    guessCC = typ.endswith(cc)
        fo.write(f"{j['id']}\t{cc}\t{name}\t{prob}\t{','.join(titles)}\t{','.join(fn)}\t{','.join(ln)}\t{guessCC}\n") 
fo.close()

  0%|          | 0/100000000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [55]:
cc

In [7]:
uk = 0
fn = '/projekti/mondonomo/books/thai/thai_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('thai_', 'nonthai_'), 'w')
for i, l in enumerate(open('/backup/wikidata/wiki_person.jsonl')):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'th' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["th"][0].replace(',',' ')
        if cc == 'TH':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head /projekti/mondonomo/books/thai/thai_wiki.csv

Q45209,TH,Poonsawat Kratingdaenggym,พูนสวัสดิ์ กระทิงแดงยิม
Q52006,TH,Pen-Ek Karaket,เป็นเอก การะเกตุ
Q57670,TH,Yingluck Shinawatra,ยิ่งลักษณ์ ชินวัตร
Q74861,TH,Thanin Kraiwichian,ธานินทร์ กรัยวิเชียร
Q78455,TH,Phra Chenduriyang (Piti Wathayakon),พระเจนดุริยางค์ (ปิติ วาทยะกร)
Q158861,TH,Chulalongkorn,พระบาทสมเด็จพระจุลจอมเกล้าเจ้าอยู่หัว
Q243623,TH,Samak Sunthorawet,สมัคร สุนทรเวช
Q284211,TH,Prayad Boonya,ประหยัด บุญญา
Q301290,TH,Phraya Phahonphonphayuhasena (Phot Phahonyothin),พระยาพหลพลพยุหเสนา (พจน์ พหลโยธิน)
Q304412,TH,Narisara Nuwattiwong,สมเด็จพระเจ้าบรมวงศ์เธอ เจ้าฟ้ากรมพระยานริศรานุวัดติวงศ์


In [114]:
from tqdm.notebook import tqdm
uk = 0
fn = '/backup/wikidata/hr_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('hr_', 'nonhr_'), 'w')
for l in tqdm(open('/backup/wikidata/wiki_person.jsonl'), total=10_044_571):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'hr' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["hr"][0].replace(',',' ')
        if cc == 'HR':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head $fn

  0%|          | 0/10044571 [00:00<?, ?it/s]

Q56220,HR,Ivo Sanader,Ivo Sanader
Q71774,HR,Igor Bubnjić,Igor Bubnjić
Q73376,HR,Wilhelm Ehm,Wilhelm Ehm
Q92358,HR,Tom Bunk,Tom Bunk
Q104440,HR,Vladimir Milojčić,Vladimir Milojčić
Q112081,HR,Archduke Wilhelm of Austria,Nadvojvoda Vilim Austrijski
Q114408,HR,Adolf Mussafia,Adolfo Mussafia
Q143424,HR,Vedran Mataija,Vedran Mataija
Q149090,HR,Ivan Ribar,Ivan Ribar
Q152506,HR,Nina Badrić,Nina Badrić


In [37]:
parse('George Washington')

{'type': {'fn1': 92.0, 'fn2': 0.0},
 'ccs': {'US': 83.4,
  'BR': 11.2,
  'GB': 2.7,
  'CA': 2.3,
  'PH': 0.3,
  'PE': 0.0,
  'NG': 0.0,
  'AR': 0.0,
  'ZA': 0.0,
  'VE': 0.0,
  'AU': 0.0,
  'IL': 0.0,
  'NZ': 0.0,
  'CL': 0.0,
  'NI': 0.0,
  'IN': 0.0,
  'MX': 0.0,
  'MY': 0.0,
  'ES': 0.0,
  'DE': 0.0,
  'FR': 0.0,
  'HN': 0.0,
  'CR': 0.0,
  'TZ': 0.0,
  'EC': 0.0,
  'GY': 0.0,
  'IT': 0.0,
  'NL': 0.0,
  'UM': 0.0,
  'CO': 0.0,
  'TR': 0.0,
  'IE': 0.0,
  'RO': 0.0,
  'HR': 0.0,
  'SG': 0.0,
  'BE': 0.0,
  'NO': 0.0,
  'CH': 0.0,
  'UY': 0.0,
  'ID': 0.0,
  'FI': 0.0,
  'SE': 0.0,
  'CZ': 0.0,
  'PK': 0.0,
  'PT': 0.0,
  'HK': 0.0,
  'DO': 0.0,
  'DK': 0.0,
  'JM': 0.0,
  'AT': 0.0,
  'PL': 0.0,
  'VN': 0.0,
  'PA': 0.0,
  'KE': 0.0,
  'HU': 0.0,
  'EE': 0.0,
  'BO': 0.0,
  'TT': 0.0,
  'UG': 0.0,
  'MH': 0.0,
  'BZ': 0.0,
  'AG': 0.0,
  'DM': 0.0,
  'BS': 0.0,
  'BB': 0.0,
  'LU': 0.0,
  'SI': 0.0,
  'VC': 0.0,
  'KN': 0.0,
  'MC': 0.0,
  'LR': 0.0,
  'SB': 0.0,
  'LV': 0.0,
  'AL'